In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

uvicorn 0.16.0 requires asgiref>=3.4.0, which is not installed.
uvicorn 0.16.0 requires h11>=0.8, which is not installed.
pydantic 1.9.1 requires dataclasses>=0.6; python_version < "3.7", which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
uvicorn 0.16.0 has requirement click>=7.0, but you'll have click 6.7 which is incompatible.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [5]:


def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_stock_result(code):
    return exec_to_result("select stok_fisik,right(no_so,3) depo \
from inventori_so_detail \
where no_so like 'so2112%' and kode_obat = '"+code+"' and stok_fisik > 0 order by stok_fisik desc")

def calc_FIFO_return_list(df,kode):
    rekapBarangKeluar = 0 if (len(get_BK(df,kode)) == 0) else get_BK(df,kode)[kode]
    rtrn = qty = 0
    for i, r in get_BKM(df,kode).iterrows():
        if (rekapBarangKeluar >= r['volAbs']):
            rekapBarangKeluar -= r['volAbs']
        else:
            show = r['volAbs'] - rekapBarangKeluar
            rekapBarangKeluar = 0
            print(i, r['kodeTrans'], show, r[13])
            rtrn += (show * r[13])
            qty += 0
    print(rtrn)
    return




In [20]:
h = get_stock_result('240411')
t = 55
for i in h:
    if (t >= i['stok_fisik']):
        t -= i['stok_fisik']
        i['target'] = 0
    else:
        i['target'] = i['stok_fisik'] - t
        t = 0
    
    
    
h

[{'stok_fisik': Decimal('300.00'), 'depo': '208', 'target': Decimal('245.00')},
 {'stok_fisik': Decimal('202.00'), 'depo': '206', 'target': Decimal('202.00')},
 {'stok_fisik': Decimal('24.00'), 'depo': '204', 'target': Decimal('24.00')},
 {'stok_fisik': Decimal('1.00'), 'depo': '126', 'target': Decimal('1.00')}]

NameError: name 'h' is not defined